# FOOD CLASSIFICATION PROJECT

*Pytorch project for AI Engineering Master with ProfessionAI*

## Colab Configuration

If you're testing this project on Google Colab, it could be useful for you to run the following cells

In [ ]:
# If you are running this code on Google Colab, run this:

!git clone https://github.com/Silvano315/PyTorch-CNN-for-food-image-classification-system.git

%cd PyTorch-CNN-for-food-image-classification-system
%pwd

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

## Import Libraries and initial Set-Up

In [1]:
# Import libraries

import os
import sys
import json
import random

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary
import albumentations as A
from albumentations.pytorch import ToTensorV2

from src.constants import RANDOM_SEED, DATA_PATH
from src.utils import get_paths_to_files, get_dataset_paths
from src.viz_fx import display_random_images

In [2]:
# Set Random Seed for reproducibility

os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [3]:
# Check if GPU is available and set it as device 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"This repository is connected to {str(device).upper()}")

This repository is connected to CPU


## Exploratory Data Analysis

In [4]:
# Get paths and filenames form directory dataset 

filepaths, filenames = get_paths_to_files(DATA_PATH)

print("="*80)
print("File paths:")
for i in range(0,5):
    print(filepaths[i])
print("="*80)
print("File names:")
for i in range(0,5):
    print(filenames[i])
print("="*80)

File paths:
dataset/test/Sandwich/Sandwich-Train (36)_e50a3a403c2741408f47d8bda7e28d1f.jpeg
dataset/test/Sandwich/Sandwich-Train (465)_f1e4e45cc2c846f19b6e2faefe8f4600.jpeg
dataset/test/Sandwich/Sandwich-Train (1162)_8f665882c03c496eba53724221b04846.jpeg
dataset/test/Sandwich/Sandwich-Train (578)_3b7f8599e1b4472f9db6fa7666eae82c.jpeg
dataset/test/Sandwich/Sandwich-Train (717)_4603bb6b236e48e9974d6b07dce7157c.jpeg
File names:
Sandwich-Train (36)_e50a3a403c2741408f47d8bda7e28d1f.jpeg
Sandwich-Train (465)_f1e4e45cc2c846f19b6e2faefe8f4600.jpeg
Sandwich-Train (1162)_8f665882c03c496eba53724221b04846.jpeg
Sandwich-Train (578)_3b7f8599e1b4472f9db6fa7666eae82c.jpeg
Sandwich-Train (717)_4603bb6b236e48e9974d6b07dce7157c.jpeg


In [5]:
# Access to path dirs and file names for each split (test, train, val)

dataset_paths = get_dataset_paths(DATA_PATH)

train_paths, train_names = dataset_paths['train']
test_paths, test_names = dataset_paths['test']
val_paths, val_names = dataset_paths['val']

print("="*80)
print("File paths:")
for i in range(0,5):
    print(train_paths[i])
print("="*80)
print("File names:")
for i in range(0,5):
    print(train_names[i])
print("="*80)

File paths:
dataset/train/Sandwich/Sandwich-Train (277)_436fb6c2c1fc4e91a9be68c03e8e45a4.jpeg
dataset/train/Sandwich/Sandwich-Train (161)_02ffbe626d9c4d969a256034e0617fdc.jpeg
dataset/train/Sandwich/Sandwich-Train (1368)_627dc1a3443d4750bafe0ca7d446ed0b.jpeg
dataset/train/Sandwich/Sandwich-Train (1016)_b9a1f780c2b54c4a8af769240dfbddb9.jpeg
dataset/train/Sandwich/Sandwich-Train (318)_778f0227997d4e50b16153d8516e5088.jpeg
File names:
Sandwich-Train (277)_436fb6c2c1fc4e91a9be68c03e8e45a4.jpeg
Sandwich-Train (161)_02ffbe626d9c4d969a256034e0617fdc.jpeg
Sandwich-Train (1368)_627dc1a3443d4750bafe0ca7d446ed0b.jpeg
Sandwich-Train (1016)_b9a1f780c2b54c4a8af769240dfbddb9.jpeg
Sandwich-Train (318)_778f0227997d4e50b16153d8516e5088.jpeg


In [ ]:
# Visualize n random images 

fig, axes = display_random_images(filepaths, n=25)
plt.show() 

In [ ]:
# Visualize n random images from a chosen split dataset (train_paths, test_paths, val_paths)

chosen_split = train_paths

fig, axes = display_random_images(chosen_split, n=25)
plt.show() 